<<日本語>>\
Inkscapeで作成したSVGファイルが入ったフォルダーをZipファイルにまとめてからプログラムを実行し、そのZipファイルをアップロードしてください。

※ 注意\
変換前のフォルダーにSVG以外のファイルが含まれていても、処理後に生成されるZipファイルにはSVGファイルのみが含まれます。SVG以外のファイルは自動的に除外されます。


<< Engrish >>

Please compress the folder containing SVG files created with Inkscape into a ZIP file, then run the program and upload the ZIP file.

Note:
Even if the original folder contains files other than SVG files, only SVG files will be included in the ZIP file generated after processing. Non-SVG files are automatically excluded.


In [ ]:
from google.colab import files
import zipfile
import os
import shutil
from IPython.core.magics import namespace
from lxml import etree

def clean_inkscape_svg(file_path,save_path):
  #SVGファイルを解析 - Parsing SVG files
  tree = etree.parse(file_path)
  root = tree.getroot()

  #消去したい名前空間のリストを作る - Create a list of namespaces you want to delete
  namespaces = {
      'inkscape' : 'http://www.inkscape.org/namespaces/inkscape',
      'sodipodi' : 'http://sodipodi.sourceforge.net/DTD/sodipodi-0.dtd'
  }

  #独自属性を全削除 - Delete all custom attributes
  for el in root.iter():
    for attr in list(el.attrib):
      if attr.startswith('sodipodi') or attr.startswith('inkscape'  ):
        del el.attrib[attr]

  #独自タグを削除 - Remove custom tags
  for tag in ['{%(sodipodi)s}namedview','{%(inkscape)s}grid','{%(inkscape)s}guide']:
    for el in root.findall('.//' + tag%namespaces):
      el.getparent().remove(el)

  # 非表示要素を削除（display:none / visibility:hidden） - Remove hidden elements (display:none / visibility:hidden)
  for el in list(root.iter()):
    style = el.attrib.get("style", "")
    if "display:none" in style or "visibility:hidden" in style:
        parent = el.getparent()
        if parent is not None:
            parent.remove(el)


  #綺麗になったSVGを保存する - Save the cleaned-up SVG
  tree.write(save_path,encoding='UTF-8',pretty_print=True,xml_declaration=True)
  print(file_path + " has been cleaned up🧹")

#変換したいファイルをZipファイルにしてアップロード - Upload the file you want to convert as a Zip file.
uploaded = files.upload()
base,ext = os.path.splitext(list(uploaded.keys())[0])
conversion_path = './conversion_files'
new_path = './newfolder'
os.makedirs(conversion_path, exist_ok=True)
os.makedirs(new_path, exist_ok=True)

#アップしたZipファイルを解凍 - Extract the uploaded ZIP file
zip_path = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(conversion_path)
  print("Thawing complete✨")

#解凍後のファイルを読み込み - Load the file after decompression
file_list = os.listdir(conversion_path)

# ファイルを掃除する - Clean up files
for root_dir, dirs, files_ in os.walk(conversion_path):
    for file in files_:
        if file.lower().endswith('.svg'):
            current_file = os.path.join(root_dir, file)
            new_file = os.path.join(new_path, file)
            clean_inkscape_svg(current_file, new_file)


#掃除後ファイルをzipファイルに圧縮 - Compress the files into a zip file after cleaning
new_zip_file = base + '_cleaned'
shutil.make_archive(new_zip_file, 'zip', new_path)
print("Compression complete❄️")

#Zipファイルをダウンロード - Download the ZIP file
files.download(new_zip_file + '.zip')
